In [13]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Cargamos los conjuntos de datos (Movies y Credits)
# Movies
movies_df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")

# Credits
credits_df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

# Mostrar las primeras filas de cada DataFrame
print("Información de películas:")
print(movies_df.head())
print("\nCréditos de películas:")
print(credits_df.head())

Información de películas:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                

In [20]:
movies_df.info()
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
# Conexión a la base de datos
conn = sqlite3.connect('movies_database.db')

# Guardar los DataFrames en tablas separadas
movies_df.to_sql('movies', conn, if_exists='replace', index=False)
credits_df.to_sql('credits', conn, if_exists='replace', index=False)

# Unir las tablas con SQL
query = """
    SELECT m.id AS movie_id, m.title, m.overview, m.genres, m.keywords, c.cast, c.crew
    FROM movies m
    JOIN credits c ON m.title = c.title
"""

# Ejecutar la consulta SQL y guardar el resultado en un DataFrame
merged_df = pd.read_sql_query(query, conn)

# Mostrar las primeras filas de la tabla unificada
print("Tabla unificada:")
print(merged_df.head())

# Cerrar la conexión a la base de datos
conn.close()

Tabla unificada:
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": 

In [6]:
# Transformar las columnas que estan en formato JSON (genres, keywords, cast, crew overview)
merged_df['genres'] = merged_df['genres'].apply(lambda x: [genre['name'] for genre in json.loads(x)]) # Genres
merged_df['keywords'] = merged_df['keywords'].apply(lambda x: [keyword['name'] for keyword in json.loads(x)]) # keywords
merged_df['cast'] = merged_df['cast'].apply(lambda x: [actor['name'] for actor in json.loads(x)[:3]]) # cast
merged_df['crew'] = merged_df['crew'].apply(lambda x: [crew_member['name'] for crew_member in json.loads(x) if crew_member['job'] == 'Director']) # crew
merged_df['overview'] = merged_df['overview'].apply(lambda x: [x]) # Overview

# Mostrar las primeras filas 
print("DataFrame transformado:")
print(merged_df.head())

DataFrame transformado:
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In the 22nd century, a paraplegic Marine is d...   
1  [Captain Barbossa, long believed to be dead, h...   
2  [A cryptic message from Bond’s past sends him ...   
3  [Following the death of District Attorney Harv...   
4  [John Carter is a war-weary, former military c...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Scie

In [7]:
# Función para eleminar espacios entre palabras
def remove_spaces(names_list):
    return [name.replace(" ", "") for name in names_list]

# Aplicamos la función a las columnas 
merged_df['genres'] = merged_df['genres'].apply(remove_spaces)
merged_df['cast'] = merged_df['cast'].apply(remove_spaces)
merged_df['crew'] = merged_df['crew'].apply(remove_spaces)
merged_df['keywords'] = merged_df['keywords'].apply(remove_spaces)

print("DataFrame con columnas procesadas:")
print(merged_df.head())

DataFrame con columnas procesadas:
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In the 22nd century, a paraplegic Marine is d...   
1  [Captain Barbossa, long believed to be dead, h...   
2  [A cryptic message from Bond’s past sends him ...   
3  [Following the death of District Attorney Harv...   
4  [John Carter is a war-weary, former military c...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure

In [8]:
# Función para combinar y limpiar los elementos 
def combine_and_clean(row):
    combined_tags = []
    for col in ['genres', 'cast', 'crew', 'keywords']:
        combined_tags.extend(row[col])
    tags_str = ', '.join(combined_tags)
    return tags_str.replace(",", " ")

# Crear la columna 'tags' combinando y limpiando los elementos de las columnas anteriores
merged_df['tags'] = merged_df.apply(combine_and_clean, axis=1)

# Mostrar las primeras filas del DataFrame con la nueva columna 'tags'
print("DataFrame con columna 'tags' combinada y limpia:")
print(merged_df[['title', 'tags']].head())

DataFrame con columna 'tags' combinada y limpia:
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                                tags  
0  Action  Adventure  Fantasy  ScienceFiction  Sa...  
1  Adventure  Fantasy  Action  JohnnyDepp  Orland...  
2  Action  Adventure  Crime  DanielCraig  Christo...  
3  Action  Crime  Drama  Thriller  ChristianBale ...  
4  Action  Adventure  ScienceFiction  TaylorKitsc...  


In [12]:
# Vectorizar la columna Tags con Term Frequency-Inverse Document Frequency

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(merged_df['tags'])

# Cálculo de la similitud de coseno
similarity = cosine_similarity(vectors)

# Crear la funcion de recomendación 
def recommend(movie):
    # Obtener el índice de la película de entrada
    movie_index = merged_df[merged_df["title"] == movie].index[0]
    
    # Calcular las distancias de similitud
    distances = similarity[movie_index]
    
    # Obtener las 5 películas más similares (excluyendo la película de entrada)
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Imprimir las películas recomendadas
    print("Películas recomendadas para", movie, ":")
    for i in movie_list:
        print(merged_df.iloc[i[0]]['title'])

# Uso de la función de recomendación (John Carter)
recommend("John Carter")

Películas recomendadas para John Carter :
Mission to Mars
My Favorite Martian
Miss Julie
Spaced Invaders
The Last Days on Mars


cambio 
